In [1]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv(r"D:\shalini\HS_DataClean_Nov.csv")

In [4]:
# Step 1: Read the config.json file
with open("D:\\Healthscore Codes\\healthscore_dashboard_config.json", 'r') as file:
    "D:\\Healthscore Codes\\healthscore_dashboard_config.json"
    config_data = json.load(file)

# Step 2: Extract config1 and config2 from the file
display_name_mapping = config_data['display_name_mapping']
level_1_metrics = config_data['level_1_metrics']
level_2_metrics = config_data['level_2_metrics']
level_3_metrics = config_data['level_3_metrics']
level_4_metrics = config_data['level_4_metrics']
level_5_metrics = config_data['level_5_metrics']
feature_groups = config_data['feature_groups']


# Step 3: Verify the contents
print(display_name_mapping)
print(level_1_metrics)
print(level_2_metrics)
print(level_3_metrics)
print(level_4_metrics)
print(level_5_metrics)
print(feature_groups)


{'DR_Visits': 'Visits', 'Unique Visitors': 'Visitors', 'Page Views': 'Page Views', 'Average Time Spent on Page': 'Average Time Spent (mins)', 'DR_New Visitors': 'New Visitors', 'DR_Server Errors': 'Server Errors', 'DR_Overall Chat': 'Visits to Chat', 'DR_continuing chat from new chat page': 'Continuing chat from new chat page', 'DR_Message Us': 'Visits to Chat (Bot)', 'DR_Agent Chat': 'Visits to Agent Chat (Skip bot)', 'DR_Self Service Initiated': 'Visits to Self Service Features', 'DR_CoxApp | Visits with Chat and No Self Service Initiated': 'Visits with Chat but No Self Service', 'DR_CoxApp | Visits with No Self Service and No Chat': 'Visits with Neither Self Service nor Chat', 'DR_CoxApp | Visits not logged in': 'Not Logged-in Visits', 'DR_Login Page Visits': 'Login Page Visits', 'DR_Successful logins': 'Login Success Rate', 'DR_Auto Logins': 'Share of Auto Login (Remember/Biometrics)', 'DR_Clicks to MFA Verification': 'with TSV', 'DR_Clicks on Forgot User ID': 'User ID Recovery', '

In [5]:
df['display_names'] = df['Metrics'].map(display_name_mapping)

In [6]:
df['level_1'] = df['display_names'].apply(lambda x: x if x in level_1_metrics else None)
df['level_2'] = df['display_names'].apply(lambda x: x if x in level_2_metrics else None)
df['level_3'] = df['display_names'].apply(lambda x: x if x in level_3_metrics else None)
df['level_4'] = df['display_names'].apply(lambda x: x if x in level_4_metrics else None)
df['level_5'] = df['display_names'].apply(lambda x: x if x in level_5_metrics else None)

In [7]:
feature_column = 'Unknown'
df['Feature'] = feature_column
for group, metrics in feature_groups.items():
   df['Feature'] = np.where(df['Metrics'].isin(metrics), group, df['Feature'])

In [8]:
def move_row(df, from_idx, to_idx):
    row = df.iloc[from_idx]
    df = df.drop(from_idx).reset_index(drop=True)
    df = pd.concat([df.iloc[:to_idx], row.to_frame().T, df.iloc[to_idx:]]).reset_index(drop=True)
    return df

df = move_row(df, 50, 46)
df = move_row(df, 51, 47)
df = move_row(df, 52, 48)
df = move_row(df, 53, 49)
df = move_row(df, 164, 158)
df = move_row(df, 165, 159)
df = move_row(df, 180, 174)
df = move_row(df, 181, 175)
df = move_row(df, 218, 206)
df = move_row(df, 219, 207)
df = move_row(df, 220, 208)
df = move_row(df, 221, 209)
df = move_row(df, 280, 268)
df = move_row(df, 281, 269)
df = move_row(df, 282, 270)
df = move_row(df, 283, 271)


In [9]:
df['unique_identifier'] = df['Feature'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))

In [10]:
def generate_hierarchy_id_grouped_by_os(df):
    df['Hierarchy_ID'] = None

   
    df['Feature_Order'] = df.groupby('Operating System Type')['unique_identifier'].transform(lambda x: pd.Series(range(len(x)), index=x.index))

    grouped = df.groupby(['Operating System Type', 'unique_identifier'], group_keys=False)
    results = []

   
    for (os_name, unique_id), group in grouped:
        current_ids = {'level_1': 0, 'level_2': 0, 'level_3': 0, 'level_4': 0, 'level_5': 0}  

        group = group.sort_values('Feature_Order')  

        for index, row in group.iterrows():
            if pd.isna(row['display_names']):
                continue  

            if not pd.isna(row['level_1']):  
                current_ids['level_1'] += 1
                current_ids['level_2'] = 0  
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}"

            elif not pd.isna(row['level_2']):  
                current_ids['level_2'] += 1
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}"

            elif not pd.isna(row['level_3']):  
                current_ids['level_3'] += 1
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}"

            elif not pd.isna(row['level_4']):  
                current_ids['level_4'] += 1
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}"

            elif not pd.isna(row['level_5']):  
                current_ids['level_5'] += 1
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}.{current_ids['level_5']}"

        results.append(group)

   
    df = pd.concat(results, ignore_index=True)

    
    #df = df[df['Display_Names'].notna()]

    
    cols = [col for col in df.columns if col != 'Hierarchy_ID' and col != 'Feature_Order'] + ['Hierarchy_ID']
    return df[cols]


df = generate_hierarchy_id_grouped_by_os(df)


In [11]:
df = generate_hierarchy_id_grouped_by_os(df)

def assign_parent_id(row):
    if not pd.isna(row['level_5']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_4']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_3']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_2']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_1']): 
        return None
    else:
        return None


df['Parent_ID'] = df.apply(assign_parent_id, axis=1)


In [12]:
df['metric_id'] = range(1, len(df) + 1)

In [13]:
df['metric_sequence_num'] = df.groupby('Operating System Type').cumcount() + 1

In [14]:
def calculate_level_no(row):
   
    if pd.isna(row['Hierarchy_ID']) or pd.isna(row['Parent_ID']):
        return 1
    
   
    return row['Hierarchy_ID'].count('.') + 1


df['level_no'] = df.apply(calculate_level_no, axis=1)


In [15]:
feature_names = df['Feature'].unique()
feature_mapping = {feature_name: idx + 1 for idx, feature_name in enumerate(feature_names)}

df['feature_id'] = df['Feature'].map(feature_mapping)


In [16]:
df['create_dt'] = pd.to_datetime('today').normalize().date()

In [17]:
df = df.rename(columns={'Parent_ID': 'parent_id_old'})

In [18]:
def calculate_parent_id(row, df):
    
    if pd.isna(row['Hierarchy_ID']):
        return None
   
    hierarchy_parts = row['Hierarchy_ID'].rsplit('.', 1)
    if len(hierarchy_parts) > 1:
       
        parent_hierarchy = hierarchy_parts[0]
        
        parent_row = df[df['Hierarchy_ID'] == parent_hierarchy]
        if not parent_row.empty:
            return parent_row['metric_id'].values[0]

    return None

df['parent_id'] = df.apply(calculate_parent_id, axis=1, df=df)


df.loc[df['Hierarchy_ID'].isna(), 'parent_id'] = None


In [19]:
df = df.rename(columns={
    'Metrics': 'metrics',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Hierarchy_ID': 'hierarchy_id',
})

In [20]:
first_table_columns = ['metric_id', 'metrics','operating_system_type','display_names', 'feature_name', 'unique_identifier', 'level_1', 'level_2', 'level_3', 'level_4','level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'level_no', 'feature_id', 'parent_id','create_dt' ]
first_table_df = df[first_table_columns]

In [21]:
#first_table_df.to_csv('hs_newfeature.csv', index = False)

In [22]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]
print(date_columns)
print(len(date_columns))

['29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-1

In [23]:
second_table_columns1 = ['metric_id', 'create_dt', 'metrics', 'operating_system_type']
second_table_columns = second_table_columns1 + date_columns
second_table_df = df[second_table_columns]

In [24]:
filtered_date_columns = second_table_df.columns[4:].to_list()
print(filtered_date_columns)

['29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024', '06-11-2024', '07-11-2024', '08-1

In [25]:
id_vars = ['metric_id', 'create_dt']

In [26]:
second_table_df_melted = second_table_df.melt(id_vars = id_vars, value_vars = filtered_date_columns, var_name = 'Date', value_name = 'Value')

In [27]:
second_table_df_melted.shape

(41670, 4)

In [28]:
#second_table_df_melted.to_csv('hs_all_feature_tab2.csv', index = False)

In [42]:
sample_df = df[df['feature_name'].isin(['SMART HELP'])]

In [29]:
def calculate_percentage(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Create a copy of the DataFrame to avoid setting on a view
    df = df.copy()
    
    # Iterate through each row and calculate the percentage for each DATE column
    for idx, row in df.iterrows():
        # Iterate through all the DATE columns
        for date_col in datecolumns:
            # Get the numerator (DATE value)
            numerator = row[date_col]
            
            # Get the parent hierarchy ID
            parent_id = row[parentid]
            
            # Check if the parent is empty
            if pd.isna(parent_id) or parent_id == "":
                # If the PARENTID is empty, keep the original DATE value
                percentage = numerator  # Assuming 100% when no parent is found
            else:
                # Find the row where HIERARCHYID == parent_hierarchyid to get the parent's DATE value
                parent_row = df[df[metricid] == parent_id]
                
                # If the parent exists, get the parent's DATE value for the current DATE column
                if not parent_row.empty:
                    denominator = parent_row[date_col].values[0]
                else:
                    denominator = 0  # In case no parent is found, avoid division by zero
                
                # Calculate the percentage
                if denominator != 0:
                    percentage = (numerator / denominator) * 100
                else:
                    percentage = 0  # To handle division by zero if no parent found
                
            # Round to 3 decimal places
            percentage = round(percentage, 3)
            
            # Add the percentage to the DataFrame in a new column
            percentage_column_name = f"{date_col} PERCENTAGE"
            if percentage_column_name not in df.columns:
                df[percentage_column_name] = None  # Initialize the column safely
            
            # Set the percentage for the current row and current date column using .loc
            df.loc[idx, percentage_column_name] = percentage

    return df


In [30]:
percentage_df = calculate_percentage(df, date_columns, 'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 5)
percentage_df.head(5)

,metrics,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,display_names,level_1,level_2,level_3,level_4,level_5,feature_name,unique_identifier,hierarchy_id,parent_id_old,metric_id,metric_sequence_num,level_no,feature_id,create_dt,parent_id,29-08-2024 PERCENTAGE,30-08-2024 PERCENTAGE,31-08-2024 PERCENTAGE,01-09-2024 PERCENTAGE,02-09-2024 PERCENTAGE,03-09-2024 PERCENTAGE,04-09-2024 PERCENTAGE,05-09-2024 PERCENTAGE,06-09-2024 PERCENTAGE,07-09-2024 PERCENTAGE,08-09-2024 PERCENTAGE,09-09-2024 PERCENTAGE,10-09-2024 PERCENTAGE,11-09-2024 PERCENTAGE,12-09-2024 PERCENTAGE,13-09-2024 PERCENTAGE,14-09-2024 PERCENTAGE,15-09-2024 PERCENTAGE,16-09-2024 PERCENTAGE,17-09-2024 PERCENTAGE,18-09-2024 PERCENTAGE,19-09-2024 PERCENTAGE,20-09-2024 PERCENTAGE,21-09-2024 PERCENTAGE,22-09-2024 PERCENTAGE,23-09-2024 PERCENTAGE,24-09-2024 PERCENTAGE,25-09-2024 PERCENTAGE,26-09-2024 PERCENTAGE,27-09-2024 PERCENTAGE,28-09-2024 PERCENTAGE,29-09-2024 PERCENTAGE,30-09-2024 PERCENTAGE,01-10-2024 PERCENTAGE,02-10-2024 PERCENTAGE,03-10-2024 PERCENTAGE,04-10-2024 PERCENTAGE,05-10-2024 PERCENTAGE,06-10-2024 PERCENTAGE,07-10-2024 PERCENTAGE,08-10-2024 PERCENTAGE,09-10-2024 PERCENTAGE,10-10-2024 PERCENTAGE,11-10-2024 PERCENTAGE,12-10-2024 PERCENTAGE,13-10-2024 PERCENTAGE,14-10-2024 PERCENTAGE,15-10-2024 PERCENTAGE,16-10-2024 PERCENTAGE,17-10-2024 PERCENTAGE,18-10-2024 PERCENTAGE,19-10-2024 PERCENTAGE,20-10-2024 PERCENTAGE,21-10-2024 PERCENTAGE,22-10-2024 PERCENTAGE,23-10-2024 PERCENTAGE,24-10-2024 PERCENTAGE,25-10-2024 PERCENTAGE,26-10-2024 PERCENTAGE,27-10-2024 PERCENTAGE,28-10-2024 PERCENTAGE,29-10-2024 PERCENTAGE,30-10-2024 PERCENTAGE,31-10-2024 PERCENTAGE,01-11-2024 PERCENTAGE,02-11-2024 PERCENTAGE,03-11-2024 PERCENTAGE,04-11-2024 PERCENTAGE,05-11-2024 PERCENTAGE,06-11-2024 PERCENTAGE,07-11-2024 PERCENTAGE,08-11-2024 PERCENTAGE,09-11-2024 PERCENTAGE,10-11-2024 PERCENTAGE,11-11-2024 PERCENTAGE,12-11-2024 PERCENTAGE,13-11-2024 PERCENTAGE,14-11-2024 PERCENTAGE,15-11-2024 PERCENTAGE,16-11-2024 PERCENTAGE,17-11-2024 PERCENTAGE,18-11-2024 PERCENTAGE,19-11-2024 PERCENTAGE,20-11-2024 PERCENTAGE,21-11-2024 PERCENTAGE,22-11-2024 PERCENTAGE,23-11-2024 PERCENTAGE,24-11-2024 PERCENTAGE,25-11-2024 PERCENTAGE,26-11-2024 PERCENTAGE
0,DR_Visits to Billing Home,Apple iOS,71742.0,76236.0,54132.0,52596.0,54447.0,68414.0,70355.0,79002.0,78610.0,50904.0,47500.0,59885.0,60599.0,66866.0,75881.0,78958.0,52352.0,48231.0,60364.0,57000.0,63905.0,64500.0,71753.0,49102.0,46906.0,58086.0,61623.0,65367.0,64902.0,74267.0,56022.0,49393.0,69232.0,77165.0,66744.0,73629.0,73180.0,53129.0,47358.0,57406.0,58604.0,65363.0,70718.0,73814.0,50310.0,42931.0,59605.0,62231.0,65649.0,67679.0,73278.0,49194.0,46554.0,58498.0,60820.0,61559.0,64139.0,66553.0,45236.0,40903.0,53844.0,55602.0,62178.0,65295.0,80962.0,55882.0,53873.0,60230.0,61201.0,61253.0,69508.0,73999.0,50243.0,43774.0,54835.0,56204.0,64712.0,75234.0,81910.0,50202.0,45884.0,57871.0,58212.0,65289.0,60150.0,69650.0,48374.0,43920.0,58120.0,59662.0,Billing Section Visits,Billing Section Visits,None,None,Non

In [46]:
percentage_df.shape

(463, 198)

In [47]:
id_columns = ['create_dt','metric_id', 'feature_id', 'metric_sequence_num', 'level_no','parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type' ]

In [48]:
# List of columns that contain "PERCENTAGE"
percentage_columns = [col for col in percentage_df.columns if 'PERCENTAGE' in col]

# Now percentage_columns contains all the column names that have "PERCENTAGE"
print(percentage_columns)

['29-08-2024 PERCENTAGE', '30-08-2024 PERCENTAGE', '31-08-2024 PERCENTAGE', '01-09-2024 PERCENTAGE', '02-09-2024 PERCENTAGE', '03-09-2024 PERCENTAGE', '04-09-2024 PERCENTAGE', '05-09-2024 PERCENTAGE', '06-09-2024 PERCENTAGE', '07-09-2024 PERCENTAGE', '08-09-2024 PERCENTAGE', '09-09-2024 PERCENTAGE', '10-09-2024 PERCENTAGE', '11-09-2024 PERCENTAGE', '12-09-2024 PERCENTAGE', '13-09-2024 PERCENTAGE', '14-09-2024 PERCENTAGE', '15-09-2024 PERCENTAGE', '16-09-2024 PERCENTAGE', '17-09-2024 PERCENTAGE', '18-09-2024 PERCENTAGE', '19-09-2024 PERCENTAGE', '20-09-2024 PERCENTAGE', '21-09-2024 PERCENTAGE', '22-09-2024 PERCENTAGE', '23-09-2024 PERCENTAGE', '24-09-2024 PERCENTAGE', '25-09-2024 PERCENTAGE', '26-09-2024 PERCENTAGE', '27-09-2024 PERCENTAGE', '28-09-2024 PERCENTAGE', '29-09-2024 PERCENTAGE', '30-09-2024 PERCENTAGE', '01-10-2024 PERCENTAGE', '02-10-2024 PERCENTAGE', '03-10-2024 PERCENTAGE', '04-10-2024 PERCENTAGE', '05-10-2024 PERCENTAGE', '06-10-2024 PERCENTAGE', '07-10-2024 PERCENTAGE',

In [49]:
percent_df = percentage_df[id_columns + percentage_columns]

,create_dt,metric_id,feature_id,metric_sequence_num,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,29-08-2024 PERCENTAGE,30-08-2024 PERCENTAGE,31-08-2024 PERCENTAGE,01-09-2024 PERCENTAGE,02-09-2024 PERCENTAGE,03-09-2024 PERCENTAGE,04-09-2024 PERCENTAGE,05-09-2024 PERCENTAGE,06-09-2024 PERCENTAGE,07-09-2024 PERCENTAGE,08-09-2024 PERCENTAGE,09-09-2024 PERCENTAGE,10-09-2024 PERCENTAGE,11-09-2024 PERCENTAGE,12-09-2024 PERCENTAGE,13-09-2024 PERCENTAGE,14-09-2024 PERCENTAGE,15-09-2024 PERCENTAGE,16-09-2024 PERCENTAGE,17-09-2024 PERCENTAGE,18-09-2024 PERCENTAGE,19-09-2024 PERCENTAGE,20-09-2024 PERCENTAGE,21-09-2024 PERCENTAGE,22-09-2024 PERCENTAGE,23-09-2024 PERCENTAGE,24-09-2024 PERCENTAGE,25-09-2024 PERCENTAGE,26-09-2024 PERCENTAGE,27-09-2024 PERCENTAGE,28-09-2024 PERCENTAGE,29-09-2024 PERCENTAGE,30-09-2024 PERCENTAGE,01-10-2024 PERCENTAGE,02-10-2024 PERCENTAGE,03-10-2024 PERCENTAGE,04-10-2024 PERCENTAGE,05-10-2024 PERCENTAGE,06-10-2024 PERCENTAGE,07-10-2024 PERCENTAGE,08-10-2024 PERCENTAGE,09-10-2024 PERCENTAGE,10-10-2024 PERCENTAGE,11-10-2024 PERCENTAGE,12-10-2024 PERCENTAGE,13-10-2024 PERCENTAGE,14-10-2024 PERCENTAGE,15-10-2024 PERCENTAGE,16-10-2024 PERCENTAGE,17-10-2024 PERCENTAGE,18-10-2024 PERCENTAGE,19-10-2024 PERCENTAGE,20-10-2024 PERCENTAGE,21-10-2024 PERCENTAGE,22-10-2024 PERCENTAGE,23-10-2024 PERCENTAGE,24-10-2024 PERCENTAGE,25-10-2024 PERCENTAGE,26-10-2024 PERCENTAGE,27-10-2024 PERCENTAGE,28-10-2024 PERCENTAGE,29-10-2024 PERCENTAGE,30-10-2024 PERCENTAGE,31-10-2024 PERCENTAGE,01-11-2024 PERCENTAGE,02-11-2024 PERCENTAGE,03-11-2024 PERCENTAGE,04-11-2024 PERCENTAGE,05-11-2024 PERCENTAGE,06-11-2024 PERCENTAGE,07-11-2024 PERCENTAGE,08-11-2024 PERCENTAGE,09-11-2024 PERCENTAGE,10-11-2024 PERCENTAGE,11-11-2024 PERCENTAGE,12-11-2024 PERCENTAGE,13-11-2024 PERCENTAGE,14-11-2024 PERCENTAGE,15-11-2024 PERCENTAGE,16-11-2024 PERCENTAGE,17-11-2024 PERCENTAGE,18-11-2024 PERCENTAGE,19-11-2024 PERCENTAGE,20-11-2024 PERCENTAGE,21-11-2024 PERCENTAGE,22-11-2024 PERCENTAGE,23-11-2024 PERCENTAGE,24-11-2024 PERCENTAGE,25-11-2024 PERCENTAGE,26-11-2024 PERCENTAGE
0,2025-01-16,1,1,1,1,NaN,BL,BIL LING,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,71742.0,76236.0,54132.0,52596.0,54447.0,68414.0,70355.0,79002.0,78610.0,50904.0,47500.0,59885.0,60599.0,66866.0,75881.0,78958.0,52352.0,48231.0,60364.0,57000.0,63905.0,64500.0,71753.0,49102.0,46906.0,58086.0,61623.0,65367.0,64902.0,74267.0,56022.0,49393.0,69232.0,77165.0,66744.0,73629.0,73180.0,53129.0,47358.0,57406.0,58604.0,65363.0,70718.0,73814.0,50310.0,42931.0,59605.0,62231.0,65649.0,67679.0,73278.0,49194.0,46554.0,58498.0,60820.0,61559.0,64139.0,66553.0,45236.0,40903.0,53844.0,55602.0,62178.0,65295.0,80962.0,55882.0,53873.0,60230.0,61201.0,61253.0,69508.0,73999.0,50243.0,43774.0,54835.0,56204.0,64712.0,75234.0,81910.0,50202.0,45884.0,57871.0,58212.0,65289.0,60150.0,69650.0,48374.0,43920.0,58120.0,59662.0
1,2025-01-16,2,1,2,2,1.0,BL,BIL LING,DR_Billing Make a Payment,Make Payments,Apple iOS,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.15,34.797,33.366,33.616,34.85,38.247,40.542,46.293,37.254,35.23,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.03,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.04,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.73,36.827
2,2025-01-16,3,1,3,3,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,0.004,0.0,0.005,0.005,0.0,0.0,0.0,0.0,0.0,0.006,0.0,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.004,0.0,0.0,0.0,0.0,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,0.003,0.0,0.0,0.0,0.004,0.0,0.0,0.0,0.0,0.0,0.0

In [51]:
# Rename columns to remove the word "PERCENTAGE"
percent_df.columns = [col.replace(" PERCENTAGE", "") if "PERCENTAGE" in col else col for col in percent_df.columns]

In [58]:
percent_df.shape

(463, 101)

In [164]:
def generate_new_rows_fixed(df, hierarchy_id_col, parent_id_col):
    """
    Generate new rows for every unique 'display_names' and calculate results efficiently.

    Args:
    - df (pd.DataFrame): Input DataFrame.
    - hierarchy_id_col (str): Name of the Hierarchy_ID column.
    - parent_id_col (str): Name of the Parent_ID column.

    Returns:
    - pd.DataFrame: Updated DataFrame with new rows.
    """
    # Extract date columns
    date_cols = date_columns

    # Step 1: Create "Both" rows
    both_rows = (
        df.groupby("display_names")[date_cols]
        .sum()
        .reset_index()
        .assign(
            operating_system_type="Both",
            metric_id=lambda x: "Both_ID_" + (x.index + 1).astype(str),
            Hierarchy_ID=None,
            Parent_ID=None,
        )
    )

    # Step 2: Initialize list for new rows
    new_rows = []

    for _, row in both_rows.iterrows():
        display_name = row["display_names"]

        # Filter iOS and Android rows for the same display_name
        ios_rows = df[
            (df["display_names"] == display_name)
            & (df["operating_system_type"] == "Apple iOS")
        ]
        android_rows = df[
            (df["display_names"] == display_name)
            & (df["operating_system_type"] == "Google Android")
        ]

        # Calculate numerator
        numerator = ios_rows[date_cols].sum().values + android_rows[date_cols].sum().values

        # Add new row with the numerator
        formula_row = row.copy()
        formula_row.update(dict(zip(date_cols, numerator)))
        new_rows.append(formula_row)

        # If Parent_ID exists, calculate denominator and result
        parent_ids = df.loc[
            (df["display_names"] == display_name) & df[parent_id_col].notna(), parent_id_col
        ].unique()

        for parent_id in parent_ids:
            # Find parent rows using the Hierarchy_ID
            parent_rows = df[df[hierarchy_id_col] == parent_id]

            if not parent_rows.empty:
                ios_parent_rows = parent_rows[
                    parent_rows["operating_system_type"] == "Apple iOS"
                ]
                android_parent_rows = parent_rows[
                    parent_rows["operating_system_type"] == "Google Android"
                ]

                # Calculate denominator
                denominator = (
                    ios_parent_rows[date_cols].sum().values
                    + android_parent_rows[date_cols].sum().values
                )

                # Result formula
                result = numerator + denominator * 100

                # Add new row for the calculated result
                formula_row = row.copy()
                formula_row[parent_id_col] = parent_id
                formula_row.update(dict(zip(date_cols, result)))
                new_rows.append(formula_row)
    new_rows_df = pd.DataFrame(new_rows)

    # Combine the original DataFrame with "Both" rows and new formula rows
    final_df = pd.concat([df, both_rows, pd.DataFrame(new_rows)], ignore_index=True)
    return new_rows_df

In [173]:
def calculate_both(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Initialize an empty list to store the new rows
    new_rows = []
   

    # Iterate through each row to generate new rows
    for idx, row in df.iterrows():
        # Check if 'parent_id_old' is None, NaN, or empty
        if pd.isna(row[parentid]) or row[parentid] == None or row[parentid] == '':
            # Handle the case where parent_id_old is missing (second function logic)
            display_name = row['display_names']
            display_name_df = df[df['display_names'] == display_name]
            
            # Create a new row for 'Both' operating system type
            #new_row = {'display_names': display_name, 'operating_system_type': 'Both', 'parent_id_old': None}
            new_row = row.to_dict()
            
            # Sum the date columns for iOS and Android
            for date_column in datecolumns:  # Assuming date columns are passed in `datecolumns`
                ios_sum = display_name_df[display_name_df['operating_system_type'] == 'Apple iOS'][date_column].sum()
                android_sum = display_name_df[display_name_df['operating_system_type'] == 'Google Android'][date_column].sum()
                
                # Add the sum of both iOS and Android
                new_row[date_column] = ios_sum + android_sum
                
            new_row['operating_system_type'] = 'Both'
            
            # Append the new row to the list of new rows
            new_rows.append(new_row)
        
        else:
            # Handle the case where parent_id_old is not missing (first function logic)
            ios_df = df[df['operating_system_type'] == 'Apple iOS']
            android_df = df[df['operating_system_type'] == 'Google Android']

            # Assuming 'metricid' and 'parentid' are the columns for Hierarchy_id and Parent_id
            ios_hierarchy_id = ios_df.loc[ios_df.index == idx, metricid].values[0] if idx in ios_df.index else None
            ios_parent_id = ios_df.loc[ios_df.index == idx, parentid].values[0] if idx in ios_df.index else None

            android_hierarchy_id = android_df.loc[android_df.index == idx, metricid].values[0] if idx in android_df.index else None
            android_parent_id = android_df.loc[android_df.index == idx, parentid].values[0] if idx in android_df.index else None

            # Initialize a dictionary to store the calculated percentages for the current row
            new_row = row.to_dict()

            # Iterate through all the DATE columns
            for date_col in datecolumns:
                # Get the numerator and denominator for both iOS and Android
                numerator_ios = df[(df[metricid] == ios_hierarchy_id)]
                numerator_android = df[(df[metricid] == android_hierarchy_id)]
                denominator_ios = df[(df[metricid] == ios_parent_id)]
                denominator_android = df[(df[metricid] == android_parent_id)]

                if not numerator_ios.empty:
                    numerator_ios_value = numerator_ios[date_col].values[0]
                else:
                    numerator_ios_value = 0

                if not denominator_ios.empty:
                    denominator_ios_value = denominator_ios[date_col].values[0]
                else:
                    denominator_ios_value = 0

                if not numerator_android.empty:
                    numerator_android_value = numerator_android[date_col].values[0]
                else:
                    numerator_android_value = 0

                if not denominator_android.empty:
                    denominator_android_value = denominator_android[date_col].values[0]
                else:
                    denominator_android_value = 0

                # Calculate the percentage
                if (denominator_ios_value + denominator_android_value) != 0:
                    result = (numerator_ios_value + numerator_android_value) / (denominator_ios_value + denominator_android_value)
                    result = result * 100
                else:
                    result = numerator_ios_value + numerator_android_value

                # Add the calculated result to the new row for the current date column
                new_row[date_col] = round(result, 3)

            # Set the operating_system_type to "Both"
            new_row['operating_system_type'] = 'Both'

            # Add the new row to the list of new rows
            new_rows.append(new_row)

    # Convert the list of new rows into a DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    
    # If the second logic was executed, drop duplicates
    #new_rows_df = new_rows_df.drop_duplicates()
    new_rows_df = new_rows_df.drop_duplicates(subset=['display_names', 'operating_system_type'], keep='first')
    #half_index = len(new_rows_df) // 2
    #new_rows_df = new_rows_df.iloc[:half_index]
    
    # Append the new rows to the original DataFrame
    df = pd.concat([df, new_rows_df], ignore_index=True)

    return new_rows_df


In [174]:
both_df = calculate_both(df, date_columns, 'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)      # Show 100 rows
both_df.head()

,metrics,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,display_names,level_1,level_2,level_3,level_4,level_5,feature_name,unique_identifier,hierarchy_id,parent_id_old,metric_id,metric_sequence_num,level_no,feature_id,create_dt,parent_id
0,DR_Visits to Billing Home,Both,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000,Billing Section Visits,Billing Section Visits,None,None,None,None,BIL LING,BL,Apple iOS_BL_1,None,1,1,1,1,2025-01-17,NaN
1,DR_Billing Make a Payment,Both,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.150,34.797,33.366,33.616,34.850,38.247,40.542,46.293,37.254,35.230,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.030,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.040,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.730,36.827,Make Payments,None,Make Payments,None,None,None,BIL LING,BL,Apple iOS_BL_1.1,Apple iOS_BL_1,2,2,2,1,2025-01-17,1.0
2,DR_Client Errors on Make Payment,Both,0.004,0.000,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.004,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.003,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.003,0.000,0.005,0.000,0.000,0.000,0.004,0.004,0.000,0.000,0.006,0.000,0.000,Client Errors on Payment Page,None,None,Client Errors on Payment Page,None,None,BIL LING,BL,Apple iOS_BL_1.1.1,Apple iOS_BL_1.1,3,3,3,1,2025-01-17,2.0
3,DR_Server Errors on Make Payment,Both,0.004,0.009,0.015,0.010,0.000,0.004,0.012,0.006,0.012,0.000,0.000,0.010,0.005,0.000,0.003,0.003,0.010,0.000,0.000,0.005,0.004,0.016,0.016,0.006,0.000,0.005,0.025,0.000,0.008,0.000,0.009,0.

In [175]:
both_df.shape

(181, 108)

In [62]:
both_df['metric_id'] = 'Both_ID_' + (both_df.index + 1).astype(str)
both_df.head()

,metrics,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,display_names,level_1,level_2,level_3,level_4,level_5,feature_name,unique_identifier,hierarchy_id,parent_id_old,metric_id,metric_sequence_num,level_no,feature_id,create_dt,parent_id
0,DR_Visits to Billing Home,Both,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000,Billing Section Visits,Billing Section Visits,None,None,None,None,BIL LING,BL,Apple iOS_BL_1,None,Both_ID_1,1,1,1,2025-01-16,NaN
1,DR_Billing Make a Payment,Both,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.150,34.797,33.366,33.616,34.850,38.247,40.542,46.293,37.254,35.230,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.030,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.040,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.730,36.827,Make Payments,None,Make Payments,None,None,None,BIL LING,BL,Apple iOS_BL_1.1,Apple iOS_BL_1,Both_ID_2,2,2,1,2025-01-16,1.0
2,DR_Client Errors on Make Payment,Both,0.004,0.000,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.004,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.003,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.003,0.000,0.005,0.000,0.000,0.000,0.004,0.004,0.000,0.000,0.006,0.000,0.000,Client Errors on Payment Page,None,None,Client Errors on Payment Page,None,None,BIL LING,BL,Apple iOS_BL_1.1.1,Apple iOS_BL_1.1,Both_ID_3,3,3,1,2025-01-16,2.0
3,DR_Server Errors on Make Payment,Both,0.004,0.009,0.015,0.010,0.000,0.004,0.012,0.006,0.012,0.000,0.000,0.010,0.005,0.000,0.003,0.003,0.010,0.000,0.000,0.005,0.004,0.016,0.016,0.006,0.000,0.005,0.025,0.

In [63]:
both_cal_df = both_df[id_columns + date_columns]
both_cal_df.head()

,create_dt,metric_id,feature_id,metric_sequence_num,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,2025-01-16,Both_ID_1,1,1,1,NaN,BL,BIL LING,DR_Visits to Billing Home,Billing Section Visits,Both,93064.000,99690.000,70846.000,68476.000,70777.000,89457.000,91825.000,102981.000,102789.000,66804.000,61675.000,77547.000,78493.000,86857.000,97900.000,102689.000,68247.000,63041.000,77821.000,73506.000,82820.000,83588.000,93325.000,63873.000,60709.000,74423.000,79316.000,85112.000,84457.000,96598.000,72554.000,63698.000,89212.000,101101.000,87238.000,97925.000,96309.000,69725.000,61658.000,74383.000,76292.000,85505.000,92385.000,96333.000,65765.000,55555.000,76985.000,80619.000,85310.000,88308.000,95437.000,64272.000,60384.000,76148.000,79110.000,79923.000,83441.000,86559.000,59192.000,53204.000,69949.000,72277.000,80683.000,85194.000,107366.000,73527.000,70262.000,78308.000,79580.000,80010.000,90858.000,96496.000,65867.000,57008.000,70899.000,72458.000,84140.000,97421.000,106566.000,65666.000,59653.000,74638.000,75190.000,84863.000,79320.000,91369.000,63344.000,56984.000,74961.000,77062.000
1,2025-01-16,Both_ID_2,1,2,2,1.0,BL,BIL LING,DR_Billing Make a Payment,Make Payments,Both,39.419,43.499,37.287,36.775,34.426,33.727,36.383,40.112,44.150,34.797,33.366,33.616,34.850,38.247,40.542,46.293,37.254,35.230,34.206,33.795,36.201,39.623,43.802,35.459,34.578,33.168,33.101,36.107,38.709,41.861,37.869,35.112,35.078,38.011,37.516,40.449,44.269,37.688,36.030,34.909,35.492,38.652,42.855,48.124,40.302,39.559,35.739,36.602,39.516,42.353,45.578,38.332,36.785,35.406,34.931,38.207,40.648,43.618,36.175,34.543,32.637,33.925,38.961,44.553,47.148,39.131,36.358,35.446,36.658,39.195,43.040,46.359,38.306,37.186,33.907,33.909,39.021,42.745,46.442,37.831,35.982,34.577,35.439,38.907,39.862,44.307,38.167,36.004,33.730,36.827
2,2025-01-16,Both_ID_3,1,3,3,2.0,BL,BIL LING,DR_Client Errors on Make Payment,Client Errors on Payment Page,Both,0.004,0.000,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.004,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.003,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.003,0.000,0.005,0.000,0.000,0.000,0.004,0.004,0.000,0.000,0.006,0.000,0.000
3,2025-01-16,Both_ID_4,1,4,3,2.0,BL,BIL LING,DR_Server Errors on Make Payment,Server Errors on Payment Page,Both,0.004,0.009,0.015,0.010,0.000,0.004,0.012,0.006,0.012,0.000,0.000,0.010,0.005,0.000,0.003,0.003,0.010,0.000,0.000,0.005,0.004,0.016,0.016,0.006,0.000,0.005,0.025,0.000,0.008,0.000,0.009,0.012,0.000,0.010,0.004,0.007,0.012,0.015,0.006,0.000,0.010,0.000,0.000,0.003,0.010,0.018,0.005,0.022,0.008,0.014,0.009,0.016,0.023,0.024,0.000,0.004,0.008,0.017,0.012,0.000,0.000,0.005,0

In [64]:
third_table_df = pd.concat([percent_df, both_cal_df], ignore_index=True)


In [65]:
third_table_df.shape

(644, 101)

In [66]:
#third_table_df.to_csv(r'D:\shalini\hs_table3_new.csv', index = False)

In [68]:
third_table_melted_df = pd.melt(third_table_df, id_vars=id_columns, 
                     value_vars=date_columns, 
                     var_name='Date', 
                     value_name='Value')

In [69]:
third_table_melted_df.shape

(57960, 13)

In [70]:
third_table_melted_df.dtypes

create_dt                 object
metric_id                 object
feature_id                 int64
metric_sequence_num        int64
level_no                   int64
parent_id                float64
unique_identifier         object
feature_name              object
metrics                   object
display_names             object
operating_system_type     object
Date                      object
Value                     object
dtype: object

In [71]:
# Convert 'create_dt' and 'Date' to datetime
third_table_melted_df['create_dt'] = pd.to_datetime(third_table_melted_df['create_dt'], errors='coerce')
third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')

# Convert 'feature_id', 'metric_sequence_num', 'level_no', 'parent_id' to integers
third_table_melted_df['feature_id'] = third_table_melted_df['feature_id'].astype('Int64')  # 'Int64' to handle missing values (NaN)
third_table_melted_df['metric_sequence_num'] = third_table_melted_df['metric_sequence_num'].astype('Int64')
third_table_melted_df['level_no'] = third_table_melted_df['level_no'].astype('Int64')
third_table_melted_df['parent_id'] = third_table_melted_df['parent_id'].astype('Int64')

# Convert 'metric_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type' to strings
third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype(str)
third_table_melted_df['unique_identifier'] = third_table_melted_df['unique_identifier'].astype(str)
third_table_melted_df['feature_name'] = third_table_melted_df['feature_name'].astype(str)
third_table_melted_df['metrics'] = third_table_melted_df['metrics'].astype(str)
third_table_melted_df['display_names'] = third_table_melted_df['display_names'].astype(str)
third_table_melted_df['operating_system_type'] = third_table_melted_df['operating_system_type'].astype(str)

# Convert 'Value' to float
third_table_melted_df['Value'] = pd.to_numeric(third_table_melted_df['Value'], errors='coerce')

C:\Users\gokul.02\AppData\Local\Temp\ipykernel_11176\2699120582.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')


In [72]:
#Calculate 1st and 3rd quartiles
quartiles = third_table_melted_df.groupby(['operating_system_type', 'display_names'])['Value'].quantile([0.25, 0.75]).unstack()
quartiles.reset_index(inplace=True)
quartiles.columns.name = None  # Remove the index name
quartiles.columns = ['operating_system_type', 'display_names', '1st Quartile', '3rd Quartile']
#Calculate IQR
quartiles['IQR'] = quartiles['3rd Quartile'] - quartiles['1st Quartile']

In [73]:
quartiles.shape

(543, 5)

In [74]:
quartiles_iqr_df = pd.merge(third_table_df, quartiles, on=['display_names', 'operating_system_type'], how='outer')

In [75]:
quartiles_iqr_df.shape

(647, 104)

In [76]:
quartiles_iqr_df.head()

,create_dt,metric_id,feature_id,metric_sequence_num,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR
0,2025-01-16,186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850
1,2025-01-16,Both_ID_186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850
2,2025-01-16,420,15.0,184.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.5,60.0,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.0,65.0,70.833,58.824,57.895,55.263,53.488,50.0,48.276,45.714,44.444,60.465,60.0,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525
3,2025-01-16,187,15.0,187.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,11.009,14.286,9.783,9.541,21.182,1.351,0.0,2.273,1.333,0.0,0.0,0.0,0.0,1.515,1.163,0.0,0.0,0.0,0.0,0.0,0.0,10.39,9.195,9.474,16.667,18.519,15.73,13.83,13.333,11.429,11.765,22.222,17.708,22.549,28.409,22.353,22.222,37.624,23.171,26.829,26.471,27.619,26.027,26.136,22.368,24.691,29.07,25.0,22.449,25.974,17.143,24.419,20.325,27.381,27.941,21.739,23

In [77]:

def calculate_upper_lower(df, parent_column='parent_id'):
    # Ensure parent_column is present in the dataframe
    if parent_column not in df.columns:
        raise ValueError(f"{parent_column} not found in the dataframe")
    
    # Define the function to apply to each row
    def calculate_row(row):
        if pd.isna(row[parent_column]) or row[parent_column] == '':
            # Apply 1.25 logic for rows where parent_id is null, empty, or None
            upper = row['3rd Quartile'] + (1.25 * row['IQR'])
            lower = row['1st Quartile'] - (1.25 * row['IQR'])
        else:
            # Apply 0.75 logic for other rows
            upper = row['3rd Quartile'] + (0.75 * row['IQR'])
            lower = row['1st Quartile'] - (0.75 * row['IQR'])
        
        return pd.Series({'Upper': upper, 'Lower': lower})
    
    # Apply the function row by row
    df[['Upper', 'Lower']] = df.apply(calculate_row, axis=1)

    # Round the Upper column to 1 decimal place
    df['Upper'] = df['Upper'].round(1)

    return df


In [78]:
upper_lower_df = calculate_upper_lower(quartiles_iqr_df)
#upper_lower_df =upper_lower_df[['display_names','operating_system_type','1st Quartile','3rd Quartile', 'IQR', 'Upper', 'Lower']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show 100 rows
upper_lower_df.head()

,create_dt,metric_id,feature_id,metric_sequence_num,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower
0,2025-01-16,186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.8,40.098875
1,2025-01-16,Both_ID_186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.8,40.098875
2,2025-01-16,420,15.0,184.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.5,60.0,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.0,65.0,70.833,58.824,57.895,55.263,53.488,50.0,48.276,45.714,44.444,60.465,60.0,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525,88.7,33.398562
3,2025-01-16,187,15.0,187.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,11.009,14.286,9.783,9.541,21.182,1.351,0.0,2.273,1.333,0.0,0.0,0.0,0.0,1.515,1.163,0.0,0.0,0.0,0.0,0.0,0.0,10.39,9.195,9.474,16.667,18.519,15.73,13.83,13.333,11.429,11.765,22.222,17.708,22.549,28.409,22.353,22.222,37.624,23.171,26.829,26.471,27.619,26.027,26.136,22.368,24.691,29.07,25.0,2

In [79]:
upper_lower_df.shape

(647, 106)

In [80]:
import numpy as np
import pandas as pd

def calculate_last7_and_30_days(df):
    # Extract the date columns (skip the non-date columns like 'display_names' and 'operating_system_type')
    datecolumn = [col for col in df.columns if '-' in col]
    
    # Exclude the latest date column (the first one)
    last_7_columns = datecolumn[-8:-1]  # Get the last 7 columns excluding the latest date
    last_30_columns = datecolumn[-31:-1]

    # Extract the date part and convert them to datetime objects
    date_objects = [pd.to_datetime(col.split()[0], format='%d-%m-%Y') for col in datecolumn]

    # Get the column corresponding to the latest date
    latest_date = max(date_objects)
    latest_date_column = datecolumn[date_objects.index(latest_date)]

    # Create a new column 'Yesterday' with the values from the latest date column
    df.loc[:, 'Yesterday'] = df[latest_date_column]
    
    # Ensure numeric columns before performing mean calculation
    df[last_7_columns] = df[last_7_columns].apply(pd.to_numeric, errors='coerce')
    df[last_30_columns] = df[last_30_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the mean for each row across the last 7 and 30 date columns
    df.loc[:, 'last_7_days'] = df[last_7_columns].mean(axis=1)
    df.loc[:, 'last_30_days'] = df[last_30_columns].mean(axis=1)

    # Replace 0 and NaN values in 'last_7_days' and 'last_30_days' with NaN to avoid division by zero
    df['last_7_days'] = df['last_7_days'].replace(0, np.nan)
    df['last_30_days'] = df['last_30_days'].replace(0, np.nan)

    # Calculate the percentage change for 'Last 7 Days' with a check for NaN
    df.loc[:, '% Change Last 7 Days'] = np.where(
        df['last_7_days'].isna(), 0, 
        (df['Yesterday'] - df['last_7_days']) / df['last_7_days'] * 100
    )

    # Calculate the percentage change for 'Last 30 Days' with a check for NaN
    df.loc[:, '% Change Last 30 Days'] = np.where(
        df['last_30_days'].isna(), 0, 
        (df['Yesterday'] - df['last_30_days']) / df['last_30_days'] * 100
    )

    # Round the percentage changes to 1 decimal place
    df.loc[:, '% Change Last 7 Days'] = df['% Change Last 7 Days'].round(1)
    df.loc[:, '% Change Last 30 Days'] = df['% Change Last 30 Days'].round(1)
    
    return df


In [81]:
last_7_and_30_days_df  = calculate_last7_and_30_days(upper_lower_df)
#last_7_and_30_days_df = last_7_and_30_days_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'parent_id','Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
last_7_and_30_days_df.head(5)

,create_dt,metric_id,feature_id,metric_sequence_num,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days
0,2025-01-16,186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.790,65.116,51.250,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.000,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.000,60.000,54.545,55.357,51.351,59.459,55.000,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.8,40.098875,66.216,58.289571,59.206200,13.598365,11.839638
1,2025-01-16,Both_ID_186,15.0,186.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.790,65.116,51.250,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.000,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.000,60.000,54.545,55.357,51.351,59.459,55.000,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.8,40.098875,66.216,58.289571,59.206200,13.598365,11.839638
2,2025-01-16,420,15.0,184.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.500,60.000,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.000,65.000,70.833,58.824,57.895,55.263,53.488,50.000,48.276,45.714,44.444,60.465,60.000,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525,88.7,33.398562,59.375,53.120714,58.331800,11.773723,1.78839
3,2025-01-16,187,15.0,187.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,11.009,14.286,9.783,9.541,21.182,1.351,0

In [82]:
last_7_and_30_days_df.shape

(647, 111)

In [83]:
#last_7_and_30_days_df.to_csv(r'D:\shalini\hs_table4_new.csv', index = False)